In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## langchain Traceing

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [3]:
# data ingestion -- from the website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/observability")
loader 

In [6]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='\n\n\n\n\nObservability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monitoring chartsDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal tracesLog retriever 

In [ ]:
## if it is huge documents we have to split it into chunks
#load the data ---> docs---> divide into chunks --- > Vectors Embeddings ---> Vectors Store Db
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)



In [8]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monito

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import FAISS
# converting into vector store

vectorstoredb = FAISS.from_documents(documents, embeddings)


In [10]:
vectorstoredb

In [ ]:
## query from thevector store db
query = ("angSmith makes this easy with the wrap_openai (Python) or wrapOpenAI")
result = vectorstoredb.similarity_search(query)
result[0].page_content

'5. Trace OpenAI callsâ€‹\nThe first thing you might want to trace is all your OpenAI calls. LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI (TypeScript) wrappers.\nAll you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly.'

In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [23]:
### retrieval chain, Document Chain 

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the question based on the context provided. If the answer is not in the context, say "I don't know".
<context>
{context}
</context>

"""
)

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context provided. If the answer is not in the context, say "I don\'t know".\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D87A6F1E80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D87A6C35F0>, root_client=<openai.OpenAI object at 0x000001D87A6AFFB0>, root_async_client=<openai.AsyncOpenAI object at 0x000001D87A6AED50>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))


In [ ]:
import langchain
langchain.debug = True

In [36]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "What is wrap_openai and how does LangSmith make it easier to use?",
    "context": [Document(page_content="LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI,All you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly")]

})
    

[chain/start] [chain:stuff_documents_chain] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context>] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] s] Exiting Chain run with output:
{
  "output": "LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI,All you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly"
}
[chain/end] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context>] s] Exiting Chain run with output:
{
  "context": "LangSmith mak

'To make it easy to use OpenAI, you can modify your code to use the wrap_openai (Python) or wrapOpenAI client instead of using the OpenAI client directly.'

However we want the documents to frist come from the retriver we just set up that way we can use this retriver to dynamically select the most revelent documents and pass those in for given questuon

In [37]:
## Input ---> Retriver ----> vectorstoredb

vectorstoredb

In [41]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain

create_retrieval_chain(retriever,document_chain)


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D87A0607A0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context provided. If the answer is not in the context, say "I don\'t know".\n<context>\n{context}\n</context>\n\n'

In [45]:
## get the response from the LLM

## Define the retrieval chain if not already defined
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input":"What is wrap_openai and how does LangSmith make it easier to use?"})
response['answer']

[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": "What is wrap_openai and how does LangSmith make it easier to use?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "What is wrap_openai and how does LangSmith make it easier to use?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "What is wrap_openai and how does LangSmith make it easier to use?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": "What is wrap_openai and how does LangSmith make it easier to use?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Wh

"The tutorial is about tracing OpenAI calls using LangSmith's observability SDK."

In [46]:
response

{'input': 'What is wrap_openai and how does LangSmith make it easier to use?',
 'context': [Document(id='98a4bbec-c35e-4479-9f2b-3aef0a4f9d4e', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='5. Trace OpenAI callsâ€‹\nThe first thing you might want to trace is all your OpenAI calls. LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI (TypeScript) wrappers.\nAll you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly.'),
  Document(id='2cd0ac5a-5b8f-44f8-b4ce-7f020e8080a8', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK 

In [47]:
response ["context"]

[Document(id='98a4bbec-c35e-4479-9f2b-3aef0a4f9d4e', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='5. Trace OpenAI callsâ€‹\nThe first thing you might want to trace is all your OpenAI calls. LangSmith makes this easy with the wrap_openai (Python) or wrapOpenAI (TypeScript) wrappers.\nAll you have to do is modify your code to use the wrapped client instead of using the OpenAI client directly.'),
 Document(id='2cd0ac5a-5b8f-44f8-b4ce-7f020e8080a8', metadata={'source': 'https://docs.smith.langchain.com/observability', 'title': 'Observability Quick Start | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', 'description': 'This tutorial will get you up and running with our observability SDK by showing you how to', 'language': 'en'}, page_content='This tutorial will get you up and r